# Demo de modelo BERT para detección de entidades

En este notebook:
- Mostramos como cargar nuestros modelos usando HuggingFace
- Transformamos su output a etiquetas de entidades interpretables

Su objetivo es ejecutar los algoritmos desde cero teniendo sólamente un ambiente conda. Su público objetivo es alguien que se interese en como los modelos tratan los datos. Si el objetivo es únicamente probar el modelo en un input de manera directa ver el notebook **demo_minimalista.ipynb** (que sin embargo requiere instalación previa).

Fue probado en un notebook Linux con ambiente conda instalado, versión de python 3.9.0.

Instalación de bibliotecas

In [ ]:
# instalación de bibliotecas necesarias
!pip install numpy==1.20.0
!pip install pandas==1.4.3
!pip install scipy==1.9.1

In [ ]:
!pip3 install torch
!pip install transformers==4.23.1

Descarga de modelos

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

MODEL = 'ccarvajal/beto-prescripciones-medicas'

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForTokenClassification.from_pretrained(MODEL, num_labels=11, ignore_mismatched_sizes=True)

Para ambos tokenizer y modelos, se puede guardar una copia local usando el método .save_pretrained()

### Output del modelo

Definimos dos funciones auxiliares que nos ayudarán con el etiquetado

In [ ]:
from scipy.special import softmax
import numpy as np

def word_ids_method(text,tokenizer):
    """Método que asigna el primer token (subword) de una palabra como representante
        La etiqueta de este token será la etiqueta de la palabra
    Método integrado en tokenize_and_align_labels
    Fuente: https://huggingface.co/docs/transformers/tasks/token_classification

    Argumentos

        text: str o List[str]
            texto a tokenizar, si 
    """
    if not isinstance(text,list):
        text = text.split()
    n_words = len(text)

    tokenized_inputs = tokenizer([text], truncation=True, is_split_into_words=True)
    mask = []
    # for i in range(n_words):
    word_ids = tokenized_inputs.word_ids(batch_index=0)
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            mask.append(0)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word.
            mask.append(1)
        else:
            mask.append(0)
        previous_word_idx = word_idx
    
    return mask

def eval_text(text,tokenizer,model):
    """
    Toma un texto (lista de palabras o string), un tokenizador y modelo de HuggingFace
    Retorna el output del modelo (ids de entidades)
    """
    mask = word_ids_method(text,tokenizer)
    encoded_input = tokenizer(text,return_tensors='pt',is_split_into_words=isinstance(text,list))
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    result = np.argmax(scores,axis=1)

    return result[mask==np.array(1)]

In [ ]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"

print(text)
eval_text(text,tokenizer,model)

Tenemos ya un output de nuestro modelo con cinco entidades, sin embargo nos gustaría tener las entidades con sus respectivas etiquetas

In [ ]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

In [ ]:
def map_entities(y_pred,map_dict,return_type='list'):
    inv_map = {v: k for k, v in map_dict.items()}
    if return_type == 'list':
        return [inv_map[y] for y in y_pred]
    else:
        return np.array([inv_map[y] for y in y_pred])

In [ ]:
entidades = map_entities(eval_text(text,tokenizer,model),ner_dict)
for i, ent in enumerate(entidades):
    print("{}\t{}".format(text.split()[i],ent))

---

## Modelo para etiquetas ADMIN más finas

In [ ]:
MODEL_admin = 'ccarvajal/beto-prescripciones-medicas-ADMIN'

tokenizer_admin = AutoTokenizer.from_pretrained(MODEL_admin)
model_admin = AutoModelForTokenClassification.from_pretrained(MODEL_admin, num_labels=7, ignore_mismatched_sizes=True)

In [ ]:
admin_ner_dict = {
    'O': 0,
    'B-CANT': 1,
    'I-CANT': 2,
    'B-UND':3,
    'I-UND':4,
    'B-VIA_ADMIN': 5,
    'I-VIA_ADMIN': 6
}

In [ ]:
def get_admin(entidades:list,texto:str) -> str:
    """Retorna un substring correspondiente a aquellas entidades etiquetadas con admin y los indices donde esto ocurre"""
    indices = [i for i, ent in enumerate(entidades) if ent == 'B-ADMIN' or ent == 'I-ADMIN']
    return ' '.join([token for i, token in enumerate(texto.split(' ')) if i in indices]), indices

In [ ]:
def etiquetar(texto):
    entidades = map_entities(eval_text(texto,tokenizer,model),ner_dict)
    admin_text, indices = get_admin(entidades,texto)
    entidades_admin = map_entities(eval_text(admin_text,tokenizer_admin,model_admin),admin_ner_dict)
    for i, ent_admin in enumerate(entidades_admin):
        entidades[indices[i]] = ent_admin
    return entidades

In [ ]:
entidades = etiquetar(text)
for i, ent in enumerate(entidades):
    print("{}\t{}".format(text.split()[i],ent))

Finalmente mostramos una manera más bonita de mostrar los datos

In [ ]:
import pandas as pd

def render_pandas(ents,text_list):
    data = {'ACTIVE_PRINCIPLE':'','FORMA_FARMA':'','CANT-ADMIN':'','UND-ADMIN':'','VIA-ADMIN':'','PERIODICITY':'','DURATION':''}

    for i, ent in enumerate(ents):
        if '-ACTIVE_PRINCIPLE' in ent:
            data['ACTIVE_PRINCIPLE'] += ' ' + text_list[i]
        elif '-FORMA_FARMA' in ent:
            data['FORMA_FARMA'] += ' ' + text_list[i]
        elif '-CANT' in ent:
            data['CANT-ADMIN'] += ' ' + text_list[i]
        elif '-UND' in ent:
            data['UND-ADMIN'] += ' ' + text_list[i]
        elif '-VIA_ADMIN' in ent:
            data['VIA-ADMIN'] += ' ' + text_list[i]
        elif '-PERIODICITY' in ent:
            data['PERIODICITY'] += ' ' + text_list[i]
        elif '-DURATION' in ent:
            data['DURATION'] += ' ' + text_list[i]

    df = pd.DataFrame([data])
    return df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide(axis='index')

In [ ]:
render_pandas(entidades,text.split())